In [1]:
#pip install openpyxl plotly

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from plotly import graph_objects as go

In [3]:
# Constants
STRIKE_ZONE = {'x_min': -8.5, 'x_max': 8.5, 'y_min': 19.47, 'y_max': 40.53}
BUFFER = 2
BALL_RADIUS = 1.47
EXPECTED_COLUMNS = {'PlateLocSide', 'PlateLocHeight'}

In [4]:
def find_header_row(df_raw):
    for i in range(len(df_raw)):
        row = df_raw.iloc[i]
        if all(isinstance(val, str) for val in row.dropna()) and EXPECTED_COLUMNS.issubset(set(row.dropna().values)):
            return i
    raise ValueError("Could not find a valid header row containing PlateLocSide and PlateLocHeight.")

def is_in_zone(x, y, zone, radius=BALL_RADIUS):
    return (zone['x_min'] - radius <= x <= zone['x_max'] + radius) and \
           (zone['y_min'] - radius <= y <= zone['y_max'] + radius)

def process_pitch_data(filepath):
    df_raw = pd.read_excel(filepath, sheet_name="PitchLocations", header=None)
    header_row = find_header_row(df_raw)
    df = pd.read_excel(filepath, sheet_name="PitchLocations", skiprows=header_row)

    # Drop rows missing coordinates
    df = df.dropna(subset=["PlateLocSide", "PlateLocHeight"])

    # Compute pitch zones
    df['raw_strike'] = df.apply(lambda row: is_in_zone(row['PlateLocSide'], row['PlateLocHeight'], STRIKE_ZONE), axis=1)
    df['buffer_pitch'] = df.apply(lambda row: not row['raw_strike'] and is_in_zone(
        row['PlateLocSide'], row['PlateLocHeight'], {
            'x_min': STRIKE_ZONE['x_min'] - BUFFER,
            'x_max': STRIKE_ZONE['x_max'] + BUFFER,
            'y_min': STRIKE_ZONE['y_min'] - BUFFER,
            'y_max': STRIKE_ZONE['y_max'] + BUFFER
        }), axis=1)
    return df

def plot_strike_zone(pitches):
    fig, ax = plt.subplots(figsize=(6, 9))

    strike_rect = patches.Rectangle(
        (STRIKE_ZONE['x_min'], STRIKE_ZONE['y_min']),
        STRIKE_ZONE['x_max'] - STRIKE_ZONE['x_min'],
        STRIKE_ZONE['y_max'] - STRIKE_ZONE['y_min'],
        linewidth=2, edgecolor='black', facecolor='none', label='Strike Zone'
    )
    buffer_rect = patches.Rectangle(
        (STRIKE_ZONE['x_min'] - BUFFER, STRIKE_ZONE['y_min'] - BUFFER),
        STRIKE_ZONE['x_max'] - STRIKE_ZONE['x_min'] + 2 * BUFFER,
        STRIKE_ZONE['y_max'] - STRIKE_ZONE['y_min'] + 2 * BUFFER,
        linewidth=1, edgecolor='gray', linestyle='--', facecolor='none', label='Buffer Zone'
    )
    ax.add_patch(strike_rect)
    ax.add_patch(buffer_rect)

    for _, row in pitches.iterrows():
        if row['raw_strike']:
            color = 'green'
        elif row['buffer_pitch']:
            color = 'orange'
        else:
            color = 'red'
        ax.plot(row['PlateLocSide'], row['PlateLocHeight'], 'o', color=color)

    ax.set_xlim(-15, 15)
    ax.set_ylim(10, 50)
    ax.set_xlabel("Horizontal Position (inches)")
    ax.set_ylabel("Vertical Position (inches)")
    ax.set_title("Trackman Pitch Location Plot")
    ax.legend()
    plt.grid(True)
    plt.show()


In [5]:
file_path = "C:/Users/hudos/Downloads/Hudson ARNEY Pitch Analysis (2).xlsm"
pitches = process_pitch_data(file_path)

In [6]:
#plot_strike_zone(pitches)

In [7]:
LEFT_BATTER_IMG = "C:/Users/hudos/Downloads/lefty-batter.png"
RIGHT_BATTER_IMG = LEFT_BATTER_IMG

In [8]:
def plot_strike_zone_interactive(pitches):
    fig = go.Figure()

    # Strike zone
    fig.add_shape(
        type="rect",
        x0=STRIKE_ZONE['x_min'], x1=STRIKE_ZONE['x_max'],
        y0=STRIKE_ZONE['y_min'], y1=STRIKE_ZONE['y_max'],
        line=dict(color="black", width=2),
        name="Strike Zone"
    )

    # Buffer zone
    fig.add_shape(
        type="rect",
        x0=STRIKE_ZONE['x_min'] - BUFFER, x1=STRIKE_ZONE['x_max'] + BUFFER,
        y0=STRIKE_ZONE['y_min'] - BUFFER, y1=STRIKE_ZONE['y_max'] + BUFFER,
        line=dict(color="gray", dash="dash"),
        name="Buffer Zone"
    )

    # Determine color by umpire call
    def get_color(pitch_call):
        if isinstance(pitch_call, str) and "ballcalled" in pitch_call.lower():
            return "green"
        elif isinstance(pitch_call, str) and "strikecalled" in pitch_call.lower():
            return "red"
        return "gray"

    # Add pitch circles and hover
    for _, row in pitches.iterrows():
        x, y = row['PlateLocSide'], row['PlateLocHeight']
        color = get_color(row.get('Pitch Called'))

        fig.add_shape(
            type="circle",
            xref="x", yref="y",
            x0=x - BALL_RADIUS, x1=x + BALL_RADIUS,
            y0=y - BALL_RADIUS, y1=y + BALL_RADIUS,
            fillcolor=color, line_color=color, opacity=0.7
        )

        fig.add_trace(go.Scatter(
            x=[x], y=[y],
            mode='markers',
            marker=dict(size=5, color='rgba(0,0,0,0)'),
            hoverinfo='text',
            text=(
                f"Pitch #{row.get('PitchNo', 'N/A')}<br>"
                f"Called: {row.get('Pitch Called', 'N/A')}<br>"
                f"Batter: {row.get('Batter Side', 'N/A')}<br>"
                f"X: {x:.2f}\"<br>Y: {y:.2f}\""
            ),
            showlegend=False
        ))

    # Detect batter side for image
    batter_sides = set(pitches['Batter Side'].dropna().str.lower())
    if 'left' in batter_sides:
        fig.add_layout_image(
            dict(
                source=LEFT_BATTER_IMG,
                xref="x", yref="y",
                x=-13, y=45,
                sizex=6, sizey=30,
                xanchor="center", yanchor="top",
                layer="below"
            )
        )
    if 'right' in batter_sides:
        fig.add_layout_image(
            dict(
                source=RIGHT_BATTER_IMG,
                xref="x", yref="y",
                x=13, y=45,
                sizex=6, sizey=30,
                xanchor="center", yanchor="top",
                layer="below"
            )
        )

    fig.update_layout(
        width=700,
        height=900,
        title="Trackman Interactive Strike Zone",
        xaxis=dict(title="Horizontal Position (inches)", range=[-15, 15]),
        yaxis=dict(title="Vertical Position (inches)", range=[10, 50]),
    )

    fig.show()

In [9]:
plot_strike_zone_interactive(pitches)

In [11]:
pitches.head()
called_strikes = pitches[pitches['Pitch Called'].str.lower().str.contains('strikecalled')]

In [12]:
plot_strike_zone_interactive(called_strikes)